### **Northwind** (Gold Modeling)

In [2]:
import json
from notebookutils import mssparkutils
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from typing import Sequence, Optional
from pyspark.sql.functions import sha2, concat_ws, current_timestamp, lit, row_number
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql.types import IntegerType
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants

StatementMeta(, 63a2e530-70e5-4e93-8676-6bf7b608fe62, 4, Finished, Available, Finished)

#### **Get Silver Metadata**
Receives Silver Table Metadata from the Silver Notebook via _Data Factory Pipeline_

In [ ]:
# Parameterized Value
silver_metadata = ""

In [ ]:
# Read Pipeline Metadata
records = json.loads(silver_metadata)
results = []

#### **Gold Ingestion**

In [ ]:
for rec in records:

    # Skip tables that failed in Silver
    if rec.get("SilverStatus") != "Success":
        rec["GoldStatus"] = "Skipped"
        continue

    try:
        # Mapping Metadata
        table_name = str(rec["TableName"]).lower() 
        model_type = rec.get("Modeling", "").lower()
        key_cols = rec["KeyColumn"].split("|")
        gold_schema = str(rec["DataDomain"]).lower()  
        warehouse_name = "DW_GOLD" 
        silver_lakehouse = "LH_SILVER" 

        # Build final dimensional table name
        if model_type == "facts":
            dim_table = f"facts_{table_name}"
        elif model_type == "dimensions":
            dim_table = f"dim_{table_name}"
        else:
            dim_table = f"bridge_{table_name}"

        # Load Silver Delta Table
        df = spark.read.table(f"{gold_schema}.{table_name}")


        # Write to Warehouse using Spark Connector 
        df.write.mode("overwrite").synapsesql(f"{warehouse_name}.{gold_schema}.{dim_table}")

        # --- Logging
        rec["GoldStatus"] = "Success"
        rec["GoldTableName"] = dim_table

    except Exception as e:
        rec["GoldStatus"] = f"Failed: {str(e)[:500]}"


#### **Notebook Return Output**

In [ ]:
# Optional: pretty-print the output
print("Returning payload →", json.dumps(records, indent=2))

# Return full JSON for pipeline/logging use
mssparkutils.notebook.exit(json.dumps(records))